In [9]:
#Import libraries
import os
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END, MessagesState
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [10]:
load_dotenv(override=True)

True

In [11]:
# 1. Update the LLM initialization for LangGraph with TrueFoundry
llm = ChatOpenAI(
    model=os.getenv("LLM_MODEL_LANGCHAIN"),
    base_url=os.getenv("LLM_GATEWAY_URL"),
    api_key=os.getenv("TFY_API_KEY"),
)

In [12]:
# --- Tools Setup ---
# Tavily Search Tool for web research
tavily_tool = TavilySearch(max_results=5)
tools = [tavily_tool]

In [13]:
# --- 2. Shared Graph State ---
class ResearchState(MessagesState):
    """
    Represents the state of our multi-step research process.
    Content is passed between nodes via this state dictionary.
    """
    query: str
    search_results: str
    final_report: str

In [14]:
# --- 3. Agent Functions (Nodes) ---
def researcher_node(state: ResearchState) -> ResearchState:
    """
    NODE 1: Acts as the Researcher Agent.
    1. Uses Tavily to find web context based on the query.
    2. Uses the LLM to summarize and synthesize the raw search results.
    """
    print("--- Researcher Node: Gathering Context ---")
    query = state["query"]
    
    # 1. Tool Call: Tavily Search
    search_context = tavily_tool.invoke({"query": query})
    # Format the search results cleanly
    context_string = "\n\n".join(
        [f"Source: {r['url']}\nTitle: {r['title']}\nSnippet: {r['content'][:300]}..." for r in search_context["results"]]
    )

   # 2. LLM Call: Synthesize/Summarize
    researcher_persona = (
        "You are a Senior Web Researcher. Your goal is to gather the latest and most relevant "
        "information about the user's query and format it as a comprehensive summary. "
        "You are an expert at utilizing the Tavily web search tool to find real-time, accurate, "
        "and cited information on any given topic. Your output must be precise and well-structured."
    )
    
    researcher_instruction = (
        f"""
        TASK: Conduct an 'advanced' web search for the user's query: '{query}'. 
        Focus on recent developments and list all sources used in the final summary.
        
        The final output MUST be a single, well-structured text summary of findings, 
        using ONLY the context provided below. Expected output: A comprehensive, cited summary.

        RESEARCH CONTEXT:
        ---
        {context_string}
        ---
        """
    )
    
    researcher_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=researcher_persona),
        HumanMessage(content=researcher_instruction)
    ])
    
    summary = researcher_prompt | llm
    summary_content = summary.invoke({}).content
    
    # Update the state with the synthesized context
    return {"search_context": summary_content}

In [15]:
def writer_node(state: ResearchState) -> ResearchState:
    """
    NODE 2: Acts as the Writer Agent.
    Takes the synthesis from the Researcher and formats it into a final Markdown report.
    """
    print("--- Writer Node: Generating Final Report ---")
    
    # Get the synthesized context from the previous node's output in the state
    summary = state.get("search_context", "No context found.")
    query = state["query"]

    # LLM Call: Writer
    writer_persona = (
        "You are a Professional Technical Writer. You are a meticulous technical writer "
        "who turns raw research data into polished, production-ready documentation. "
        "Your goal is to write a final, professionally formatted markdown report based on the context provided."
    )

    writer_instruction = (
        f"""
        TASK: Based on the summary provided by the Researcher Agent, write a final report for the query: '{query}'.
        
        The report must be in **Markdown format** with a clear title (using #) and bullet points.
        The final output must be ONLY the Markdown text. Expected output: A Markdown formatted report.

        RESEARCH SUMMARY:
        ---
        {summary}
        ---
        """
    )

    writer_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=writer_persona),
        HumanMessage(content=writer_instruction)
    ])

    report_chain = writer_prompt | llm
    final_report_content = report_chain.invoke({}).content
    
    # Update the state with the final report (this will be the final output of the graph)
    return {"final_report": final_report_content}

In [16]:
# --- 4. Graph Construction ---
# Initialize the StateGraph
workflow = StateGraph(ResearchState)

# Add nodes corresponding to the agents
workflow.add_node("researcher", researcher_node)
workflow.add_node("writer", writer_node)

# Define the sequential edges (Researcher -> Writer -> END)
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "writer")
workflow.set_finish_point("writer")

# Compile the graph
app = workflow.compile()

In [17]:
# --- Function to be Wrapped by FastMCP --- this is synchronous here, but we'll make it asynchronous in the final script.
def run_research_analysis(query: str) -> str:
    """
    Executes the compiled LangGraph workflow with a user query.
    This function is what the FastMCP server will call.
    """
    initial_state = {
        "query": query, 
        "search_context": "", 
        "final_report": "", 
        "messages": [HumanMessage(content=query)]
    }
    
    # Invoke the compiled graph
    final_state = app.invoke(initial_state)
    
    # Return the final report content
    return final_state["final_report"]

In [18]:
if __name__ == "__main__":
    # Example Query
    query = "Latest developments in quantum computing hardware in 2025" 
    print(f"--- Starting LangGraph Research Flow for: {query} ---")
    result = run_research_analysis(query)
    print("\n\n=== FINAL REPORT (LangGraph Output) ===")
    print(result)

--- Starting LangGraph Research Flow for: Latest developments in quantum computing hardware in 2025 ---
--- Researcher Node: Gathering Context ---
--- Writer Node: Generating Final Report ---


=== FINAL REPORT (LangGraph Output) ===
# Latest Developments in Quantum Computing Hardware (2025)

- Currency note: My knowledge is current to October 2024. The items below summarize the state of the art through 2024 and outline concrete 2025 developments to verify. Use the “How to verify” section to check the latest claims and announcements.

## Cross-cutting 2025 themes to watch
- From physical to logical: Efforts shift from record physical qubits to sustained operation of small logical qubit sets, with emphasis on scalable error-correction cycles, active reset, and leakage management.
- Modularity and interconnects: Chiplet-based QPUs, photonic/microwave links between dies and cryostats, and early quantum networking prototypes targeting reliable inter-module entanglement.
- Integrated contro

In [19]:
#Final response
display(Markdown(result))

# Latest Developments in Quantum Computing Hardware (2025)

- Currency note: My knowledge is current to October 2024. The items below summarize the state of the art through 2024 and outline concrete 2025 developments to verify. Use the “How to verify” section to check the latest claims and announcements.

## Cross-cutting 2025 themes to watch
- From physical to logical: Efforts shift from record physical qubits to sustained operation of small logical qubit sets, with emphasis on scalable error-correction cycles, active reset, and leakage management.
- Modularity and interconnects: Chiplet-based QPUs, photonic/microwave links between dies and cryostats, and early quantum networking prototypes targeting reliable inter-module entanglement.
- Integrated control electronics: Cryo-CMOS/RF front-ends moving closer to the qubits to reduce latency, wiring count, and heat load; tighter co-design of hardware, calibration, and compilers.
- 3D integration and advanced packaging: Through-silicon vias, bump-bonded resonator chips, superconducting interposers, integrated optics for ions/atoms, and high-density vacuum/cryogenic feedthroughs.
- Materials and device quality: Incremental gains in coherence, stability, and yield via better epitaxy, isotopic purification, low-loss dielectrics, and noise mitigation.
- Practical benchmarking: Transition from raw qubit counts to application-relevant metrics (logical fidelity, cycle time, stability/uptime, cross-talk, throughput) and task-oriented benchmarks.
- Reliability and automation: Improved auto-calibration, drift compensation, and fleet operations; system uptime becomes a key buying criterion for cloud users.

## Platform-by-platform developments and 2025 checkpoints

### Superconducting circuits
- Gate-level progress: Continued pursuit of high-fidelity two-qubit gates with lower leakage and cross-talk; exploration of fluxonium and tunable-coupler designs to ease frequency crowding.
- Scaling enablers: Multi-chip modules, 3D integration, and resonator buses for larger lattices; denser cryogenic wiring and improved on-package filtering.
- Error correction: Faster, lower-latency readout and feed-forward for surface/bosonic codes; verify demonstrations of multiple interacting logical qubits and extended logical lifetimes.
- 2025 checkpoints to verify:
  - Stable multi-chip systems with uniform performance across tiles.
  - Repeated, durable logical operations (stabilizer cycles) with quantified logical error rates.
  - Reduced calibration time and improved system uptime at scale.

### Trapped ions
- Architecture: Longer chains with better mode management; shuttling architectures in microfabricated traps; integration of on-chip optics for addressing and detection.
- Speedups: Microwave- and laser-driven gates with reduced crosstalk; improved sympathetic cooling and mid-circuit measurement.
- Networking: Photonic interconnects between traps for modularity; assess entanglement rates/fidelities between modules.
- 2025 checkpoints to verify:
  - Multi-module demonstrations with routine remote entanglement.
  - Logical qubit experiments with reproducible, extended lifetimes.
  - More compact, automated laser and vacuum subsystems.

### Neutral atoms (Rydberg/tweezer arrays)
- Scaling and geometry: Large 2D/3D arrays with programmable connectivity; enhanced stability of Rydberg interactions and error mitigation.
- Digital/analog hybrid: Progress on gate-based control alongside analog simulation; improved single- and two-qubit gate fidelities over larger arrays.
- Modularity: Photonic links between arrays; integrated optics for addressing.
- 2025 checkpoints to verify:
  - High-fidelity two-qubit gates across sizable 2D arrays.
  - Early error-correction primitives on atom platforms.
  - Repeatable remote entanglement between arrays or systems.

### Photonic quantum computing
- Sources and detectors: Brighter, purer on-chip sources with active/deterministic multiplexing; scalable low-jitter SNSPD arrays; low-loss, fast reconfigurable photonics.
- Architectures: Time-domain multiplexed cluster states and programmable interferometers; hybrid matter–photon transduction for networking and memory.
- 2025 checkpoints to verify:
  - Large, low-loss cluster-state generation with verified entanglement structure.
  - Error-corrected or error-detecting photonic primitives with feed-forward.
  - Loss-tolerant architectures integrated on a single photonic platform.

### Semiconductor spin qubits (Si/SiGe, others)
- CMOS-foundry leverage: Better uniformity and yield in 2D quantum dot arrays; faster exchange gates; operation at elevated cryo temps with integrated control.
- Control stack: Cryo-CMOS multiplexing/readout; improved charge noise mitigation; spin shuttling and coupling strategies.
- 2025 checkpoints to verify:
  - Coherent control across tens of coupled spins with uniform performance.
  - Demonstrations of error-correction primitives in 2D arrays.
  - Robust operation with integrated cryo-electronics reducing wiring overhead.

### Topological approaches (e.g., Majorana)
- Device physics: Materials quality, spectroscopy, and reproducible signatures remain the focus; independent validations of non-Abelian behavior.
- 2025 checkpoints to verify:
  - Clear, reproducible braiding-like experiments vetted by multiple labs.
  - Two-qubit interactions and scaling prospects beyond single devices.

## Error correction and control in 2025
- Surface and color codes: Lower-latency stabilizer readout, mid-circuit measurement, active reset, and leakage handling; verify logical cycle times and error budgets.
- Bosonic codes (cat/GKP): Progress on autonomous stabilization and bias-preserving gates; track logical lifetimes and gate fidelities.
- Co-design: Hardware-aware compilation, crosstalk-aware scheduling, and real-time calibration; quantification of overheads for first useful logical workloads.

## Packaging, cryogenics, and infrastructure
- Cryogenics: Higher cooling power at sub-Kelvin stages, reduced vibration, denser wiring, and improved thermalization; reliability and maintenance intervals as KPIs.
- Photonics and vacuum: Integrated optics and stable UHV platforms for atoms/ions; compact laser systems with lower SWaP.
- Electromagnetics: Better shielding, filtering, and microwave components for low-noise operation; standardized interconnects and feedthroughs.

## Benchmarks and metrics that matter in 2025
- Logical metrics: Logical error rate per cycle, logical T1/T2, and sustained logical circuit depth.
- System performance: Throughput (circuits/second), scheduler latency, calibration overhead, and uptime.
- Fidelity and stability: Two-qubit gate fidelities under load, cross-talk maps, drift rates, and auto-calibration effectiveness.
- Networking: Inter-module entanglement rate and Bell-state fidelity; link stability over time.
- Resource efficiency: Power and space per logical qubit; cryo wiring density; control-channel utilization.

## Commercial and ecosystem signals to watch
- Cloud access to error-corrected prototypes and standardized logical APIs.
- Foundry partnerships and PDK maturity for superconducting/spin devices.
- Independent benchmarking results and third-party audits of system performance.
- Supply-chain robustness for cryogenics, lasers, detectors, and specialty materials.
- Pre-competitive standards for metrics, data formats, and safety in shared facilities.

## How to verify the latest (2025)
- Vendor sources: Roadmaps and technical blogs from major players (e.g., IBM Research, Google Quantum AI, Quantinuum, IonQ, Rigetti, Intel, PsiQuantum, Xanadu, Atom Computing, QuEra, Pasqal, Oxford Ionics, Universal Quantum).
- Peer-reviewed and preprints: arXiv (quant-ph, cond-mat, appl-phys) and subsequent journal publications; look for independent replications.
- Conferences and workshops: APS March Meeting, CLEO, IEEE IEDM, IEEE Quantum Week, Q2B, and platform-specific workshops for fresh hardware results.
- Independent evaluations: Community benchmarks, third-party labs, and consortium reports (e.g., QED-C) that assess reliability, performance, and practical utility.
- Procurement disclosures: Cloud service changelogs, system uptime dashboards, and specification sheets that include logical-level metrics, not only qubit counts.

## Bottom line for 2025
- Expect a continued pivot from headline qubit counts to demonstrable logical performance, modular scaling, and operational reliability.
- Verify claims through logical-level metrics, sustained system operation, and independent replication before drawing conclusions about practical readiness.